# Bibliotecas


In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import tensorflow as tf
import ee
import numpy as np
from osgeo import ogr
from osgeo import gdal
import glob,pygeoj
import rasterio
from rasterio.plot import show
from patchify import patchify, unpatchify


import json
import math
import logging
import time
from datetime import datetime, date, timedelta

import folium
from PIL import Image
from matplotlib import pyplot as plt
from IPython.display import Image

logging.getLogger('googleapicliet.discovery_cache').setLevel(logging.ERROR)

gpu_dict    = {'4090':{'GPU_AFFINTY' : 0, 'GPU_MEMORY_LIMIT_GB':12}}
sel_gpu     = '4090'
GPU_AFFINTY = gpu_dict[sel_gpu]['GPU_AFFINTY'] 
GPU_MEMORY_LIMIT_GB = gpu_dict[sel_gpu]['GPU_MEMORY_LIMIT_GB']
USER_EE_PROJECT='USER_PROJECT_ID'

try:
    ee.Initialize(project = USER_EE_PROJECT)
except:
    ee.Authenticate()
    ee.Initialize(project = USER_EE_PROJECT)

# GPU config

In [ ]:
EE_TILES = 'https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}'

print('Tensorflow Version:',tf.__version__)
print('Folium Version:',folium.__version__)

gpus = tf.config.list_physical_devices('GPU')

if gpus:
  try:
    tf.config.set_visible_devices(gpus[GPU_AFFINTY], 'GPU')
    GPU_MEMORY_LIMIT_GB = GPU_MEMORY_LIMIT_GB * 1e3
    if GPU_MEMORY_LIMIT_GB == 0:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    else:
        tf.config.set_logical_device_configuration(gpus[GPU_AFFINTY],[tf.config.LogicalDeviceConfiguration(memory_limit=GPU_MEMORY_LIMIT_GB)])
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    print(e)

In [ ]:
def create_directory(new_folder):
  ''' Check if any of the specified folder already exist'''
  if not os.path.exists(new_folder):
      print(f'lets make the directory: {new_folder}')
      os.makedirs(new_folder)
  else: return

  def check_file_exists(paths):
    """Check if any of the specified paths already exist"""
    for path in paths:
        if os.path.exists(path):
            return True
    return False

# ENV Configs

In [ ]:
# General 
GPU_AFFINITY   = gpus[GPU_AFFINTY].name
MOSAIC_VERSION = '1'
MAPBIOMAS_V    = '10'

VERSION        = '4_p5_DATA_AUGMENTATION'
MOSAIC_VERSION = '1'


GOAL_YEAR      = '2022'
GOAL_CLASS     = 'htf'
GDRIVE         = f'mb{MAPBIOMAS_V}-unet-htf-brazil_'+VERSION

FOLDER_TRAIN   = f'mb{MAPBIOMAS_V}_htf_training_samples'
FOLDER_TEST    = f'mb{MAPBIOMAS_V}_htf_eval_samples'

TRAINING_BASE  = 'training_patches_'+MOSAIC_VERSION+'_v'+SAMPLE_VERSION
EVAL_BASE      = 'eval_patches_'+MOSAIC_VERSION+'_v'+SAMPLE_VERSION

#Local paths
CURRENT_LOCAL_PATH  = f'~/Mapbiomas/modelos/mb{MAPBIOMAS_V}-unet-{GOAL_CLASS}' 

MODEL_DIR   = f'{CURRENT_LOCAL_PATH}/checkpoint/v{VERSION}'
OUTPUT_PATH = CURRENT_LOCAL_PATH+'/output/v'+VERSION
create_directory(MODEL_DIR)
create_directory(OUTPUT_PATH)


# Specify inputs (Landsat bands) to the model and the response variable.
opticalBands   = ['green','red','nir','swir1']
opticalIndices = ['NDVI','MNDWI']
BANDS          = opticalBands + opticalIndices

RESPONSE = 'supervised'
FEATURES = BANDS + [RESPONSE]

# Specify the size and shape of patches expected by the model.
KERNEL_SIZE  = 256
KERNEL_SHAPE = [KERNEL_SIZE, KERNEL_SIZE]
COLUMNS = [
  tf.io.FixedLenFeature(shape=KERNEL_SHAPE, dtype=tf.float32) for k in FEATURES
]
FEATURES_DICT = dict(zip(FEATURES, COLUMNS))

# Sizes of the training and evaluation datasets.
TRAIN_SIZE = 0
EVAL_SIZE  = 0


# Specify model training parameters.
BATCH_SIZE  = 10 
DROPOUT     = 0.3 
EPOCHS      = 50
BUFFER_SIZE = 1000 
OPTIMIZER   = 'Nadam' 
LOSS        = 'BinaryCrossentropy'
METRICS     = ['RootMeanSquaredError']

# Data Visualization

In [ ]:
baseClassV       = '4'
yearClass_class  = '2022'
yearClass_mosaic = '2022'
version_final    = '4'
classID          = 32

USER_PATH_MOSAIC = 'USER_PATH_MOSAIC'

supervised_layer  = ee.Image('projects/solved-mb10/assets/public/LANDSAT/HTF/'+yearClass_class+'-'+version_final+'_SUPERVISEDMASK_FULL').eq(classID).rename(RESPONSE)
supervisedChannel = supervised_layer.toByte().rename(RESPONSE)


image = ee.Image('projects/'+USER_EE_PROJECT+'/assets/'+USER_PATH_MOSAIC+'/mosaic_'+yearClass_mosaic).addBands(supervisedChannel)
mapid = image.getMapId({'bands': ['red', 'green', 'blue'], 'min': 11, 'max': 95})

map = folium.Map(location=[-23.0089, -43.6078],zoom_start=13)
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Planet',
    overlay=True,
    name='Mosaic composite',
  ).add_to(map)
mapid = supervisedChannel.select(RESPONSE).mask(supervisedChannel.eq(1)).getMapId({'min': 0, 'max': 1, 'palette':'red'})
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Google Earth Engine',
    overlay=True,
    name='Apicum '+yearClass_class,
  ).add_to(map)
map.add_child(folium.LayerControl())
map

In [ ]:
featureStack = ee.Image.cat([
  image.select(BANDS).unmask(0),
  image.select(RESPONSE).unmask(0)
]).float()

list = ee.List.repeat(1, KERNEL_SIZE)
lists = ee.List.repeat(list, KERNEL_SIZE)
kernel = ee.Kernel.fixed(KERNEL_SIZE, KERNEL_SIZE, lists)

arrays = featureStack.neighborhoodToArray(kernel)

In [ ]:
yearClass_geoms = '2020'
trainingPolys_v1 = ee.FeatureCollection('projects/solved-mb10/assets/public/LANDSAT/HTF/trainPolys_htf_'+yearClass_geoms+'_v1')
evalPolys_v1     = ee.FeatureCollection('projects/solved-mb10/assets/public/LANDSAT/HTF/testPolys_htf_'+yearClass_geoms+'_v1')

trainingPolys_v2 = ee.FeatureCollection('projects/solved-mb10/assets/public/LANDSAT/HTF/trainPolys_htf_'+yearClass_geoms+'_v2_update')
evalPolys_v2     = ee.FeatureCollection('projects/solved-mb10/assets/public/LANDSAT/HTF/testPolys_htf_'+yearClass_geoms+'_v2_update')

trainingPolys_v3 = ee.FeatureCollection('projects/solved-mb10/assets/public/LANDSAT/HTF/trainPolys_htf_'+str(2022)+'_v3')
evalPolys_v3     = ee.FeatureCollection('projects/solved-mb10/assets/public/LANDSAT/HTF/testPolys_htf_'+str(2022)+'_v3')

trainingPolys = trainingPolys_v1.merge(trainingPolys_v2)
evalPolys     = evalPolys_v1.merge(evalPolys_v2).merge(evalPolys_v3)

id_filter_out = ['2_00000000000000000018','2_0000000000000000001f','1_00000000000000000004','2_0000000000000000001d']
trainingPolys = trainingPolys.filter(ee.Filter.inList('system:index', id_filter_out).Not())

trainingPolys = trainingPolys.merge(trainingPolys_v3)

def geo_type(feature):
    return feature.set('geo_type', feature.geometry().type())
trainingPolys = trainingPolys.map(lambda feat: geo_type(feat))
trainingPolys = trainingPolys.filter(ee.Filter.neq('geo_type','LineString'))

print(trainingPolys.size().getInfo())
print(evalPolys.size().getInfo())

polyImage = ee.Image(0).byte().paint(trainingPolys, 1).paint(evalPolys, 2)
polyImage = polyImage.updateMask(polyImage)

mapid = polyImage.getMapId({'min': 1, 'max': 2, 'palette': ['red', 'blue']})
map = folium.Map(location=[-1.3621, -45.2738], zoom_start=5)
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Google Earth Engine',
    overlay=True,
    name='training polygons',
  ).add_to(map)
map.add_child(folium.LayerControl())
# map

# Train/Test Chips Exportation

In [ ]:
version_samples_acc = "4"

# Convert the feature collections to lists for iteration.
trainingPolysList = trainingPolys.toList(trainingPolys.size())
evalPolysList = evalPolys.toList(evalPolys.size())
# These numbers determined experimentally.
n = 20 # Number of shards in each polygon.
N = 200 # Total sample size in each polygon.
FOLDER_TEST    = 'testing_samples_FOR_DEEPLABV3'


#Add some generalism
TRAIN_SIZE = trainingPolys.size().getInfo()*N
EVAL_SIZE = evalPolys.size().getInfo()*N
print('TRAIN:'+str(TRAIN_SIZE))
print('EVAL:'+str(EVAL_SIZE))
GDRIVE = 'MB8_Apicum_v5'
# Export all the training data (in many pieces), with one task 
# per geometry.
for g in range(trainingPolys.size().getInfo()):
  geomSample = ee.FeatureCollection([])
  for i in range(n):
    sample = arrays.sample(
      region = ee.Feature(trainingPolysList.get(g)).geometry(), 
      scale = 30, 
      numPixels = N / n, # Size of the shard.
      seed = i,
      tileScale = 8
    )
    geomSample = geomSample.merge(sample)
  
  desc = TRAINING_BASE + '_g' + str(g)
  task = ee.batch.Export.table.toDrive(
    collection = geomSample,
    description = desc, 
    folder = GDRIVE+'/'+FOLDER_TRAIN+'_v'+version_samples_acc, 
    fileNamePrefix = desc,
    fileFormat = 'TFRecord',
    selectors = BANDS + [RESPONSE]
  )
  # task.start()

# Export all the evaluation data.
for g in range(evalPolys.size().getInfo()):
  geomSample = ee.FeatureCollection([])
  for i in range(n):
    sample = arrays.sample(
      region = ee.Feature(evalPolysList.get(g)).geometry(), 
      scale = 30, 
      numPixels = N / n,
      seed = i,
      tileScale = 8
    )
    geomSample = geomSample.merge(sample)
  
  desc = EVAL_BASE + '_g' + str(g)
  task = ee.batch.Export.table.toDrive(
    collection = geomSample,
    description = desc, 
    folder = GDRIVE+'/'+FOLDER_TEST+version_samples_acc, 
    fileNamePrefix = desc,
    fileFormat = 'TFRecord',
    selectors = BANDS + [RESPONSE],
  )
  # task.start()

In [ ]:
def parse_tfrecord(example_proto):
  """The parsing function.
  Read a serialized example into the structure defined by FEATURES_DICT.
  Args:
    example_proto: a serialized Example.
  Returns: 
    A dictionary of tensors, keyed by feature name.
  """
  print(FEATURES_DICT)
  return tf.io.parse_single_example(example_proto, FEATURES_DICT)



def to_tuple(inputs):
  """Function to convert a dictionary of tensors to a tuple of (inputs, outputs).
  Turn the tensors returned by parse_tfrecord into a stack in HWC shape.
  Args:
    inputs: A dictionary of tensors, keyed by feature name.
  Returns: 
    A dtuple of (inputs, outputs).
  """
  inputsList = [inputs.get(key) for key in FEATURES]
  stacked = tf.stack(inputsList, axis=0)
  # Convert from CHW to HWC
  stacked = tf.transpose(stacked, [1, 2, 0])
  return stacked[:,:,:len(BANDS)], stacked[:,:,len(BANDS):]


def get_dataset(pattern):
  """Function to read, parse and format to tuple a set of input tfrecord files.
  Get all the files matching the pattern, parse and convert to tuple.
  Args:
    pattern: A file pattern to match in a Cloud Storage bucket.
  Returns: 
    A tf.data.Dataset
  """
  # glob = tf.gfile.Glob(pattern) for tendorflow 1.x
  glob = tf.io.gfile.glob(pattern) # for tendorflow 2.x
  dataset = tf.data.TFRecordDataset(glob, compression_type='GZIP')
  dataset = dataset.map(parse_tfrecord, num_parallel_calls=5)
  dataset = dataset.map(to_tuple, num_parallel_calls=5)
  return dataset

In [ ]:
from tensorflow.keras import layers
import copy
import random
seed = 42
tf.random.set_seed(seed)

data_augmentation_model = tf.keras.Sequential([
    layers.RandomFlip("vertical", seed=seed),
    layers.RandomRotation(factor=0.2, seed=seed),
    layers.RandomTranslation(height_factor=0.1, width_factor=0.1, seed=seed)
])

def create_data_augmentation_v2(model):

    return model

def augment_fn(x, y):
    # data_augmentation_func = create_data_augmentation(seed)
    data_augmentation_func = create_data_augmentation_v2(data_augmentation_model)
    input_data = tf.concat([x, y], axis=-1)
    
    # Check if the instance contains your target class
    tensor_sum_value  = tf.math.reduce_sum(y, axis=[0,1,2])
    tensor_base_value = tf.constant([183.], dtype=tf.float32)
    greater_tensor    = tf.greater(tensor_sum_value, tensor_base_value)

    if tf.reduce_any(greater_tensor):
        augmented_data = data_augmentation_func(input_data)
        x_aug = augmented_data[:, :, :len(BANDS)]
        y_aug = augmented_data[:, :, len(BANDS):]
        return (x_aug, y_aug)
    else:
        return (x, y)
    
def is_augmented(x, y):
    tensor_sum_value  = tf.math.reduce_sum(y, axis=[0,1,2])
    tensor_base_value = tf.constant([183.], dtype=tf.float32)
    greater_tensor    = tf.greater(tensor_sum_value, tensor_base_value)
    return tf.reduce_any(greater_tensor)


def get_training_dataset_data_aug_target(SAMPLE_PATH):
    """Get the preprocessed training dataset
  Returns: 
    A tf.data.Dataset of training data.
  """
    
    glob    = SAMPLE_PATH + '/'+ TRAINING_BASE + '*'   
    dataset_bkp = get_dataset(glob)
    dataset     = get_dataset(glob)
    print('\n')
    
    
    num_samples = 31800
    
    for index in range(1):
        augmented_dataset = dataset_bkp.map(augment_fn)
        augmented_only_dataset = augmented_dataset.filter(is_augmented).take(15900)
        dataset = dataset.concatenate(augmented_only_dataset)
    
    dataset = dataset.shuffle(BUFFER_SIZE, reshuffle_each_iteration=True).batch(8).repeat().prefetch(buffer_size=tf.data.AUTOTUNE)
    return dataset

version_samples_acc = "4_p5"
TRAIN_PATH = CURRENT_LOCAL_PATH+'/train/v'+version_samples_acc
create_directory(TRAIN_PATH)
training = get_training_dataset_data_aug_target(TRAIN_PATH)
print(training.take(1))

In [ ]:
def get_eval_dataset(SAMPLE_PATH):
    glob    = SAMPLE_PATH + '/'+ EVAL_BASE + '*'
    dataset = get_dataset(glob)
    dataset = dataset.batch(1).repeat()
    return dataset

version_samples_acc = "4_p5"
EVAL_PATH = CURRENT_LOCAL_PATH+'/eval/v'+version_samples_acc
create_directory(EVAL_PATH)
evaluation = get_eval_dataset(EVAL_PATH)

In [ ]:
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt

def previewClass(epoch,log):
    counter = 0
    for batch in evaluation.shuffle(1000).take(3):
        pureImage = batch[0]
        supervised = batch[1]
        stacked = tf.transpose(pureImage[0], [0, 1, 2]).numpy()
        stackedS = tf.transpose(supervised[0], [0, 1, 2]).numpy()

        test_pred_raw = m.predict(pureImage)
        test_pred_raw = tf.transpose(test_pred_raw[0],[0, 1, 2]).numpy()
        fig = plt.figure(figsize=[12,4])
        # show original image
        fig.add_subplot(131)
        plt.imshow(stacked[:,:,0:3].astype(np.uint8), interpolation='nearest', vmin=0, vmax=255)
        fig.add_subplot(132)
        plt.imshow(stackedS[:,:,0], interpolation='nearest',cmap="gray")
        fig.add_subplot(133)
        plt.imshow(test_pred_raw[:,:,0], interpolation='nearest',cmap="gray")
        plt.show()
        counter = counter+1

# UNET

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import models
from tensorflow.keras import metrics
from tensorflow.keras import optimizers


def conv_block(input_tensor, num_filters):
	encoder = layers.Conv2D(num_filters, (3, 3), padding='same')(input_tensor)
	encoder = layers.BatchNormalization()(encoder)
	encoder = layers.Activation('relu')(encoder)
	encoder = layers.Conv2D(num_filters, (3, 3), padding='same')(encoder)
	encoder = layers.BatchNormalization()(encoder)
	encoder = layers.Activation('relu')(encoder)
	return encoder

def encoder_block_ori_unet(input_tensor, num_filters):
	encoder = conv_block(input_tensor, num_filters)
	encoder_pool = layers.MaxPooling2D((2, 2), strides=(2, 2))(encoder)
	return encoder_pool, encoder

def decoder_block(input_tensor, concat_tensor, num_filters):
	decoder = layers.Conv2DTranspose(num_filters, (2, 2), strides=(2, 2), padding='same')(input_tensor)
	decoder = layers.concatenate([concat_tensor, decoder], axis=-1)
	decoder = layers.BatchNormalization()(decoder)
	decoder = layers.Activation('relu')(decoder)
	decoder = layers.Conv2D(num_filters, (3, 3), padding='same')(decoder)
	decoder = layers.BatchNormalization()(decoder)
	decoder = layers.Activation('relu')(decoder)
	decoder = layers.Conv2D(num_filters, (3, 3), padding='same')(decoder)
	decoder = layers.BatchNormalization()(decoder)
	decoder = layers.Activation('relu')(decoder)
	return decoder

def get_model():
	inputs = layers.Input(shape=[None, None, len(BANDS)]) # 256 (shape=[256, 256, len(BANDS
	encoder0_pool, encoder0 = encoder_block_ori_unet(inputs, 64) # 128
	encoder1_pool, encoder1 = encoder_block_ori_unet(encoder0_pool, 128) # 64
	encoder2_pool, encoder2 = encoder_block_ori_unet(encoder1_pool, 256) # 32
	encoder3_pool, encoder3 = encoder_block_ori_unet(encoder2_pool, 512) # 16
	center = conv_block(encoder3_pool, 1024) # 8 center
	decoder4 = decoder_block(center, encoder3, 512) # 16
	decoder3 = decoder_block(decoder4, encoder2, 256) # 32
	decoder2 = decoder_block(decoder3, encoder1, 128) # 64
	decoder1 = decoder_block(decoder2, encoder0, 64) # 128
	dropout = layers.Dropout(DROPOUT, name="dropout", noise_shape=None, seed=None)(decoder1)


	outputs = layers.Conv2D(1, (1, 1),  activation=tf.nn.sigmoid, padding='same', kernel_initializer=tf.keras.initializers.GlorotNormal())(dropout) #tensorflow 2.x    
	model = models.Model(inputs=[inputs], outputs=[outputs])
	optimizer = tf.keras.optimizers.Nadam(0.000005, name='optimizer')

	model.compile(
		optimizer=optimizer, 
		loss=losses.get(LOSS),
		metrics=[metrics.get(metric) for metric in METRICS]
    )
	return model

# FCN-DK (FCN with dilated kernel)

In [ ]:
loaded_model = get_model()
EPOCH = 0
if EPOCH>0:
    LOADED_MODEL_DIR = f'{MODEL_DIR}/cp-00{str(EPOCH)}.keras'
    loaded_model.load_weights(LOADED_MODEL_DIR)
print(loaded_model.summary())

In [ ]:
plt.style.use("ggplot")

checkpoint_path = MODEL_DIR+"/cp-{epoch:04d}.keras"
checkpoint_dir = os.path.dirname(checkpoint_path)
create_directory(checkpoint_dir)
print(checkpoint_dir)


n = 20 # Number of shards in each polygon.
N = 200 # Total sample size in each polygon.
TRAIN_SIZE = trainingPolys.size().getInfo()*N
EVAL_SIZE = evalPolys.size().getInfo()*N

tensorboard = tf.keras.callbacks.TensorBoard(log_dir=f'output/v{VERSION}/log_model',write_images=True)
cp_callback  = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,verbose=1, save_weights_only=False,save_best_only=False,save_freq='epoch')
img_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=previewClass)
earlyStopping_callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10)



TRAIN_SIZE = 47700
EVAL_SIZE  = 13355

result = loaded_model.fit(x=training,
  epochs=100,
  initial_epoch=0, # REMEMBER TO CHANGE THIS INITIAL EPOCH PARAM, WHEN OTHER MODEL HAS BEEN LOADED
  steps_per_epoch=int(TRAIN_SIZE / BATCH_SIZE),
  verbose=1,
  shuffle=True,
  validation_data=evaluation,
  validation_steps=EVAL_SIZE,
  callbacks = [cp_callback,img_callback,tensorboard,earlyStopping_callback])

In [ ]:
import time

kernel_buffer   = [256, 256]
ROOT_PATH      = './'
MOSAIC_VERSION = '1'
mosaic_scale   = 30
GRIDS_IDS    = pygeoj.load(f'{ROOT_PATH}/GRIDS/GRID-ALLCALSSES-COL9.geojson')
GRID         = pygeoj.load(f'{ROOT_PATH}/GRIDS/GRID-ALLCALSSES-COL9-STACK.geojson')

reduced_grid = [int(feature.properties['id']) for feature in GRIDS_IDS if feature.properties['apicum'] == 1]
reduced_grid = [int(n + 1) for n in reduced_grid]
reduced_grid.sort()

# Export mosaics to GDrive

In [ ]:
def doExport(out_image_base,year,region_id, kernel_buffer, roi):
  """Run the image export task."""
  image = ee.Image('projects/'+USER_EE_PROJECT+'/assets/USER_PATH/mosaic_'+str(year))
  # Export the image, specifying scale and region.
  task = ee.batch.Export.image.toDrive(
    image          = image.select(BANDS).toFloat(),
    description    = out_image_base+'_'+str(year),
    fileNamePrefix = out_image_base+'_'+str(year), 
    folder         = 'mosaics_landsat/'+str(year),
    scale          = 30,
    region         = roi,
    fileFormat     = 'GEOTIFF',
    formatOptions  = { 
      'patchDimensions': KERNEL_SHAPE,
      'kernelSize': kernel_buffer,
      'compressed': True,
      'maxFileSize': 157286400
    }
  )
  task.start()

# Run the export.
for region in reduced_grid.sort():
    region_id = int(region.properties['id'])
    image_base_name =f'v{MOSAIC_VERSION}_L9_grid_{region_id}'
    if int(region_id):
      print('Region:',int(region_id))
      for y in range(1985, 2024): 
          doExport(image_base_name,y, kernel_buffer, region.geometry.coordinates[0])

# Prediction

# Grid

In [ ]:
def mosaic_predict(mosaic_lzw,year, region_id, output_path, version, kernel_dim, optical_bands, optical_indices, model, mosaic_scale, EPOCH):
    """Executes segmentation over mosaic data exported from EE as .geotiff

    Parameters
    ----------
    mosaic_lzw : geotiff
        Geotiff data representing the LANDSAT mosaic. The region exported must be the same as the region_id
    year : int
        The year of said geotiff
    region_id : int
        The id of the geojson grid that identifies the geotiff region
    output_path: str
        The output path for the segmented data
    version: int
        The segmentation version
    kernel_dim: int
        The dimention of the patches to be segmented
    optical_bands: list
        List of optical bands present on each geotiff
    optical_indices: list
        List of optical indices on each geotiff
    model: Tensorflow model
        The model trained
    mosaic_scale: int
        The scale of the geotiff. Tipically, for landsat the scale is 30
    EPOCH: int
        The epoch of training for the model. For identification

    Returns
    -------
    Geotiff
        A geotiff corresonding to the segmentation fo the target for the mosaic_lzw
    """

    paths_to_check = [
        f'{output_path}/{year}/e{EPOCH}/outimage_v{version}_e{EPOCH}_grid_{region_id}_{year}_lzw.tif',
        f'{output_path}/{year}/e{EPOCH}/outimage_v{version}_e{EPOCH}_grid_{region_id}_{year}_byte_lzw.tif',
    ]
    if check_file_exists(paths_to_check):
        return f'GRID {region_id} already predicted'

    with rasterio.open(mosaic_lzw, 'r') as ds:
        arr = ds.read()

    arr = np.clip(arr, 0, None)
    img_arr_original = arr.astype(np.float32)

    if img_arr_original.shape[0]> 6:
        img_arr_original = img_arr_original[1:, :, :] # delete blue band
    img_arr_original = np.nan_to_num(img_arr_original, nan=0.0) 
    print(f'ORIGINAL SHAPE: {img_arr_original.shape}')
    
    ''' MAKE THE IMAGE QUADRATIC '''
    arr_shape_xy = np.array(img_arr_original.shape[1:])
    min_dim = arr_shape_xy.min()
    index_min_dim = np.where(arr_shape_xy==min_dim)[0][0]
    if index_min_dim == 0: # [len(bands), x, y], x<y
        img_arr = img_arr_original[:, :, :min_dim] 
    elif index_min_dim ==1:  # [len(bands), x, y], x>y
        img_arr = img_arr_original[:, :min_dim, :]
    
    ''' ENSURE IMAGE DIMENSIONS ARE MULTIPLES OF kernel_dim '''
    
    pad_size = (kernel_dim - (min_dim % kernel_dim)) % kernel_dim
    
    if pad_size > 0:
        img_arr = np.pad(img_arr, ((0, 0), (0, pad_size), (0, pad_size)), mode='mean') # Pads with the edge values of array.

    disired_dims    = kernel_dim*2 
    bands           = optical_bands + optical_indices
    patches         = patchify(img_arr, (len(bands), disired_dims, disired_dims), step=kernel_dim)
    dim             = patches.shape[1]
    patch2          = patches.reshape((1, dim**2, len(bands), disired_dims, disired_dims))
    patch2_reshaped = patch2[0].reshape((dim**2, len(bands), disired_dims, disired_dims))
    patch3          = np.transpose(patch2_reshaped, [0,2,3,1])

    curr_patch      = tf.data.Dataset.from_tensor_slices(patch3).batch(1)
    with tf.device('/device:GPU:0'):
        predictions_arr = model.predict(curr_patch, batch_size=8,steps=None, verbose=1)
    patchesPerRow  = dim
    TotalPatches   = dim**2
    patchDimension = [disired_dims,disired_dims]

    counter       = 1
    rowCounter    = 1
    globalCounter = 0
    finalArray    = np.array([])
    rowArray      = np.array([])

    for raw_record in predictions_arr:
        raw_record = np.squeeze(raw_record)
        rows,cols = raw_record.shape

        limite_esquerda = kernel_dim//2
        limite_direita  = kernel_dim + (kernel_dim//2)
        limite_inferior = kernel_dim + (kernel_dim//2)
        limite_superior = kernel_dim//2
        if rowCounter == 1: # FIRST ROW
            limite_superior = 0 
        if (counter == 1) or (counter == patchesPerRow+1): # FIRST COLUMN
            limite_esquerda = 0
        if (counter == patchesPerRow+1) and rowCounter == 1: # FIRST COLUMNS ON SECUND ROW
            limite_superior = kernel_dim//2  

        if counter == patchesPerRow:  # LAST COLUMN
            limite_direita = kernel_dim * 2 

        if rowCounter == (TotalPatches/patchesPerRow) or (rowCounter == (TotalPatches/patchesPerRow)-1 and counter == patchesPerRow+1):  # LAST ROW
            limite_inferior = kernel_dim * 2
        raw_record = raw_record[limite_superior:limite_inferior,limite_esquerda:limite_direita]
        if rowCounter == 1:
            finalArray = rowArray
        if counter <= patchesPerRow:
            if counter == 1:
                rowArray = raw_record
            else:
                rowArray = np.concatenate((rowArray,raw_record), axis = 1)
            counter = counter+1
        else:
            counter = 2
            rowCounter = rowCounter+1
            if np.array_equal(finalArray,rowArray):
                finalArray = rowArray
            else:
                finalArray = np.concatenate((finalArray,rowArray),axis=0)
            rowArray = raw_record
        globalCounter = globalCounter+1
    finalArray = np.concatenate((finalArray,rowArray),axis=0)


    finalArray_padded = dynamic_slice_or_pad(arr_shape_xy, finalArray)
    print(f'finalArray_padded: {finalArray_padded.shape}\n\n')
    
    rows,cols = finalArray_padded.shape
    finalArray_padded = np.array([finalArray_padded]) 

    output_path = f'{output_path}/{year}/e{EPOCH}'
    create_directory(output_path)
    raster_uri     = output_path + '/UNET_v'+version+'grid'+str(region_id)+'_'+str(year)+'.tif'
    try:
        if not np.any(np.isnan(finalArray_padded)):
            finalArray_padded = np.round((finalArray_padded.astype(np.float32))*255).astype(np.uint8)
        raster_uri_lzw = f'{output_path}/outimage_v{version}_e{EPOCH}_grid_'+str(region_id)+'_'+str(year)+'_byte_lzw.tif'
        data_type = "uint8"
    except Exception as inst:
        print(f'ERROR: For grid {region_id} \n {inst.args}, {inst}\n Raster in float')
        raster_uri_lzw = f'{output_path}/outimage_v{version}_e{EPOCH}_grid_'+str(region_id)+'_'+str(year)+'_float_lzw.tif'
        data_type = "float32"  
    

    with rasterio.open(raster_uri,'w',
                  driver="GTiff",
                  height=rows,
                  width=cols,
                  count=1,
                  dtype=data_type,
                  crs='EPSG:4326',
                  transform=ds.transform,
                  nodata=0) as dataset:
                      dataset.write(finalArray_padded)
    dataset = gdal.Open(raster_uri, gdal.GA_Update)
    !gdal_translate -of GTiff -ot Byte -co "COMPRESS=LZW" -co "PREDICTOR=2" -co "TILED=YES" {raster_uri} {raster_uri_lzw} 
    !rm {raster_uri}
    print("C'est finiz\n\n")
    return f'GRID {region_id} predito'

In [ ]:
def dynamic_slice_or_pad(target_shape, predicted_img):

    """Add padding to the segmentation output so that it mayches the mosaic geotiff input dimentions

    Parameters
    ----------
    target_shape : list
        List with the size of total rows and columns of the input image
    predicted_img : Numpy array
        Array repreenting the segmented output

    Returns
    -------
    Numpy array
        Numpy array representing the segmentation output properly padded
    """
    target_rows, target_cols = target_shape
    pad_rows = target_rows - predicted_img.shape[0]
    pad_cols = target_cols - predicted_img.shape[1]

    if pad_rows < 0:
        predicted_img = predicted_img[:target_rows, :]
    elif pad_rows > 0:
        predicted_img = np.pad(predicted_img, ((0, pad_rows), (0, 0)), mode='constant')

    if pad_cols < 0:
        predicted_img = predicted_img[:, :target_cols]
    elif pad_cols > 0:
        predicted_img = np.pad(predicted_img, ((0, 0), (0, pad_cols)), mode='constant')
    
    return predicted_img

In [ ]:
start = time.time()
for year in range(2023,2024):
    print(f'/n/nYEAR:{year}')
    i = 0
    add_list=[]
    for region_id in reduced_grid:
        print(f'REGION ID {region_id}')
        tf.keras.backend.clear_session()
        start = time.time()
        # try:
        MOSAC_PATH    = f'{ROOT_PATH}/mosaico_landsat/{year}'
        search_pattern = os.path.join(MOSAIC_PATH, f'v{MOSAIC_VERSION}_*_L*_grid_{region_id}_{year}*.tif')
        matching_files = glob.glob(search_pattern)
        
        if matching_files:
            i = i+1
            region_mosaic_file = matching_files[0]
            mosaic_predict(region_mosaic_file, year, region_id, OUTPUT_PATH, VERSION, KERNEL_SIZE, opticalBands, opticalIndices, loaded_model, mosaic_scale)
        else:
            print(i)
            logging.error("No matching:",search_pattern)
            add_list.append(region_id)
            print("No matching:",search_pattern)

end = time.time()
print(add_list)
print('Prediction Time per year = '+str(end - start))